# Exercise 4. Clustering

In this exercise k-means clustering is used for finding 5 clusters from a 3 band Sentinel satellite image. The same image as for forest classification is used.

In [ ]:
from sklearn.cluster import KMeans
import rasterio
import numpy as np
import os
import time

In [ ]:
### File paths.
### Change the base_folder if needed

base_folder='/home/jovyan/work/geocomputing/machineLearning/data/forest'
inputImage =  os.path.join(base_folder,'T34VFM_20180829T100019_clipped_scaled.tif')
outputImage = os.path.join(base_folder,'T34VFM_20180829T100019_KMeans.tif')
satelliteImage = os.path.join(base_folder,'T34VFM_20180829T100019_clipped_scaled.tif')
forestClasses = os.path.join(base_folder,'forest_species_reclassified_clip.tif')

## 4.1 Read data and shape it to suitable form for scikit-learn.

The satellite image has 3 channels, so rasterio reads it in as 3D data cube.

For scikit-learn we reshape the data to 2D, having in dataframe one row for each pixel. Each pixel has three values, one for each band.

In [ ]:
# Read the pixel values from .tif file as dataframe
image_dataset = rasterio.open(inputImage)  
image_data = image_dataset.read()

# We have to change the data format from bands x width x height to width*height x bands
# This means that each pixel from the original dataset has own row in the result dataframe.
# Check shape of input data
print ('Dataframe original shape, 3D: ', image_data.shape)    

As a mid-step transponse the axis order, so that the bands are the last. Notice how the dataframe size changes.

In [ ]:
# First move the bands to last axis.
image_data2 = np.transpose(image_data, (1, 2, 0))
# Check again the data shape, now the bands should be last.
print ('Dataframe shape after transpose, 3D: ', image_data2.shape) 

Then reshape to 2D.

In [ ]:
pixels = image_data2.reshape(-1, 3)
print ('Dataframe shape after transpose and reshape, 2D: ', pixels.shape) 

*How many pixels there is?*

## 4.2 Calculate clusters

Find 5 clusters using 5 iterations. *Try also with for example 7 classes.*

*This takes a moment, please wait*

In [ ]:
classes = KMeans(n_clusters=5, random_state=63, max_iter=5).fit_predict(pixels)

Check the results. *How many pixels each class has?*

In [ ]:
np.unique(classes, return_counts=True)

## 4.3 Reshape results back to 2D raster

The clustering results are per pixel, so first reshape the data back to 2D raster.

In [ ]:
print ('Dataframe shape, output, 1D: ', classes.shape) 

In [ ]:
#Reshape back to 2D    
classes2D = np.reshape(classes, (image_dataset.meta['height'], image_dataset.meta['width']))
print ('Dataframe shape, output after reshape, 2D: ', classes2D.shape)  

## 4.4 Plot results

Create plots in similar way as in Exercise 2.

* Satellite image
* Clustering results
* Forest

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
%matplotlib inline
import rasterio
import numpy as np
from rasterio.plot import show
from rasterio.plot import show_hist

In [ ]:
### Help function to normalize band values and enhance contrast. Just like what QGIS does automatically
def normalize(array):
    min_percent = 2   # Low percentile
    max_percent = 98  # High percentile
    lo, hi = np.percentile(array, (min_percent, max_percent))
    return (array - lo) / (hi - lo)

In [ ]:
### Create a subplot for 4 images and plot the sentinel image 
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(20, 20))

### The Sentinel image   
sentinel = rasterio.open(satelliteImage)

### Read the bands separately and apply the normalize function to each of them to increase contrast
nir, red, green = sentinel.read(1), sentinel.read(2), sentinel.read(3)
nirn, redn, greenn = normalize(nir), normalize(red), normalize(green)
stacked = np.dstack((nirn, redn, greenn))

ax[0, 0].imshow(stacked)

### The clustering results
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","darkgreen","violet","blue"]) #,"white","black"
show(classes2D, ax=ax[0, 1], cmap=cmap, title='Clustering classes')

### The forest classification
forest_classes = rasterio.open(forestClasses)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white","orange","darkgreen","violet"])
show(forest_classes, ax=ax[1, 0], cmap=cmap, title='Forest classes')

### Plot the histogram of clustering results
show_hist(classes2D, ax=ax[1, 1], bins=[-0.5,0.5,1.5,2.5,3.5,4.5], title="Clustering histogram") #,5.5,6.5

## 4.4 Save results

Save the result to a GeoTiff file. First prepare the metadata of the new file - compared to the original file, the new file has onlyl 1 band and int32 data type.

In [ ]:
meta = image_dataset.meta
meta.update(count=1, dtype='int32')
# Save the data
with rasterio.open(outputImage, 'w', **meta) as dst:
    dst.write(classes2D, 1)